In [1]:
# TO DO:
# create printouts/logs of record processing.  muck with logger framework
    # Make sure to do error logging.

In [2]:
import json
import pandas as pd
import numpy as np

import re

from pprint import pprint

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [88]:
def parse_dollars(s):
    if type(s) != str:
        return np.nan
    
    # form one: r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"
    # form two: r"\$\s*\d+[,\.]\d{3}"
    
    # form: "$###.# billion: 
    # remove dollar signs, whitespace, and text.  
    # Multiply by 1billion
    if re.match(r"\$\s*\d{1,3}\.?\d*\s*billi?on", s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        value = float(s) * 10**9
        return value
     
    # form: "$###.# million: 
    # remove dollar signs, whitespace, and text.  
    # Multiply by 1million   
    if re.match(r"\$\s*\d{1,3}\.?\d*\s*milli?on", s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        value = float(s) * 10**6
        return value    
    
    # form: $###,###,###
    # strip dollar signs and thousands separators
    if re.match(r"\$\s*\d+[,\.]\d{3}", s, flags=re.IGNORECASE):
        s = re.sub('\$|,|\.','',s)
        value = float(s)
        return value
        
    else:
        return np.nan

In [3]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) # creates a non-destructive copy.  DON'T UNDERSTAND THIS SYNTAX
    
    # Clean alternate titles
    alt_titles = dict()
    languages = ['Arabic',
                 'Cantonese',
                 'Chinese',
                 'French',
                 'Hangul',
                 'Hebrew',
                 'Hepburn',
                 'Japanese',
                 'Literally',
                 'Mandarin',
                 'McCune–Reischauer',
                 'Polish',
                 'Revised Romanization',
                 'Romanized',
                 'Russian',
                 'Simplified',
                 'Traditional',
                 'Yiddish']

    for language in languages:
        if language in movie:
            alt_titles[language] = movie[language]
            movie.pop(language)

    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director(s)')
    change_column_name('Director', 'Director(s)')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    return movie

In [96]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
def import_source_files(wiki_file: str,
                        kaggle_file: str,
                        ratings_file: str):
    """
    Function takes three arguments, each corresponding to the name
    of a specific source csv or json file for the three types of data 
    objects we are importing.  Returns all three objects as unique
    pandas DataFrames.
    """
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_json = json.load(file)
    
    # Remove TV shows
    wiki_movies_json = [wiki_movies_json[i]\
                        for i in range(len(wiki_movies_json))\
                        if 'No. of episodes' not in wiki_movies_json[i]]
    
    # Iterate through clean movie function to tidy columns
    wiki_movies_json = [clean_movie(wiki_movies_json[i]) for i in range(len(wiki_movies_json))]
    
    # Create dataframe
    wiki_movies_df = pd.DataFrame(wiki_movies_json)
    
    # Extract all IMDB IDs from valid URls and remove records that do not contain them
    try:
        wiki_movies_df.dropna(subset=['imdb_link'], inplace=True)
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except:
        print('IMDB extraction failed.')
    
    columns_to_drop = [column\
                         for column in wiki_movies_df.columns\
                         if wiki_movies_df[column].count() == 0]   
        
    wiki_movies_df.drop(columns=columns_to_drop, inplace=True)
    
    #Regex strings for currency patterns                                   
    form_one = r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"                                       
    form_two = r"\$\s*\d+[,\.]\d{3}"
                                       
    box_office = wiki_movies_df['Box office'].dropna()
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)                                       
                                       
    wiki_movies_df['box_office'] = box_office.str.\
                                    extract(f"({form_one}|{form_two})",\
                                            flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

In [5]:
# 7. Set the three variables in Step 6 equal to the function created in Step 1.

    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    

    # Open and read the Wikipedia data JSON file.
    
    
    # 3. Write a list comprehension to filter out TV shows.
    

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.


    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
#     try:
        
#     except 

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    

    # 8. Create a variable that will hold the non-null values from the “Box office” column.

    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
   
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    

    # 12. Add the parse_dollars function.
    
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.

    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    
    # Return three variables. The first is the wiki_movies_df DataFrame


In [97]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = './data'
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [98]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = import_source_files(wiki_file=wiki_file, 
                                                           kaggle_file=kaggle_file, 
                                                           ratings_file=ratings_file)

In [99]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file
wiki_movies_df["box_office"].isnull().count()

7049

In [38]:
# test
box_office = wiki_movies_df['Box office'].dropna()
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office.head(50)

0                        $21.4 million
1                         $2.7 million
2                          $57,718,089
3                           $7,331,647
4                     $6,939,946 (USA)
9                             $855,810
10                      $195.3 million
11                       $53.2 million
12                       $15.7 million
13                       $52.1 million
14                      $244.5 million
15                         $12,626,043
17                         $19,740,070
18                        $138,697,012
20                          $1,292,323
21                        $8.2 million
22                       $15.6 million
24                          $2,070,871
25       $27.6 million (United States)
27                             $10,173
29                          $5 million
33                       $35.8 million
34                       US$ 4,212,828
37                        $2.4 million
39                            $947,306
40                       

In [60]:
# ES exploratory notes: delete me when done
form_one = r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"
contains_form_one = box_office.str.contains(pat=form_one, flags=re.IGNORECASE, na=False)
contains_form_one.head(50)

0      True
1      True
2     False
3     False
4     False
9     False
10     True
11     True
12     True
13     True
14     True
15    False
17    False
18    False
20    False
21     True
22     True
24    False
25     True
27    False
29     True
33     True
34    False
37     True
39    False
40    False
42    False
43     True
44    False
45     True
48     True
50     True
55     True
56    False
58     True
59     True
60     True
63    False
65     True
66     True
68     True
69    False
70     True
74    False
75    False
76    False
77     True
78    False
79    False
80    False
Name: Box office, dtype: bool

In [61]:
# ES exploratory notes: delete me when done
box_office[~contains_form_one].head(50)

2                           $57,718,089
3                            $7,331,647
4                      $6,939,946 (USA)
9                              $855,810
15                          $12,626,043
17                          $19,740,070
18                         $138,697,012
20                           $1,292,323
24                           $2,070,871
27                              $10,173
34                        US$ 4,212,828
39                             $947,306
40                             $715,983
42                        US$13,236,513
44                           $1,373,728
56     $6,698,361 ( United States ) [2]
63                           $2,346,150
69                           $1,372,350
74                          $14,760,451
75                    $22,424,195 (USA)
76                  $6,488,144 (US) [1]
78                           $1,823,154
79                           $1,305,887
80                             $205,068
81                             $335.000


In [62]:
# exploratory: remove form two
form_two = r"\$\s*\d+[,\.]\d{3}"
contains_form_two = box_office[~contains_form_one].\
                    str.contains(pat=form_two, flags=re.IGNORECASE, na=False)
# contains_form_two.head(50)
box_office[~contains_form_one & ~contains_form_two]

81      $335.000
1676    $520.000
2453    $282.175
2940    $414.000
3044    $621.000
4273    $161.000
Name: Box office, dtype: object

In [45]:
# ES: exploratory: Export columns to csv file for analysis and mapping
count_list = wiki_movies_df.count()
col_count_df = pd.DataFrame(count_list)
col_count_df.to_csv('data\wiki_col_count.csv')

In [11]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.count()

url                  7049
year                 7049
imdb_link            7049
title                7044
Screenplay by        2307
                     ... 
Film(s)                 1
Screen story by         1
Original work           1
Television series       1
imdb_id              7049
Length: 91, dtype: int64

In [95]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Productioncompany ',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director(s)',
 'Distributor',
 'Editor(s)',
 'Producer(s)',
 'Writer(s)',
 'Genre',
 'Theme music composer',
 'Original language(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'alt_titles',
 'Released',
 'Recorded',
 'Venue',
 'Label',
 'Area',
 'Coordinates',
 'Status',
 'Opening date',
 'Closing date',
 'Replaced',
 'Replaced by',
 'Name',
 'Attr

In [19]:
print([column\
       for column in wiki_movies_df.columns\
       if wiki_movies_df[column].count()/len(wiki_movies_df) < .01])

['Theme music composer', 'Production location(s)', 'Picture format', 'Audio format', 'Voices of', 'Followed by', 'Composer(s)', 'Created by', 'Preceded by', 'Author', 'Publisher', 'Publication date', 'Media type', 'Pages', 'ISBN', 'OCLC', 'LC Class', 'Cover artist', 'Series', 'Set in', 'Adaptation by', 'Suggested by', 'alt_titles', 'Released', 'Recorded', 'Venue', 'Label', 'Area', 'Coordinates', 'Status', 'Opening date', 'Closing date', 'Replaced', 'Replaced by', 'Name', 'Attraction type', 'Music', 'Duration', 'Also known as', 'Animation by', 'Color process', 'Characters', 'Date premiered', 'Place premiered', 'Setting', 'Original language', 'Subject', 'Text', 'Original title', 'Nationality', 'Portrayed by', 'Alias', 'Species', 'Gender', 'Family', 'Alma mater', 'Film(s)', 'Screen story by', 'Original work', 'Television series']
